In [2]:
import pandas as pd
import numpy as np
import sklearn

### Import Markdown

In [3]:
housing_data = pd.read_csv('data/2014_Housing_Market_Analysis_Data_by_Zip_Code.csv')
crime_2016_data = pd.read_csv('data/2016_Annual_Crime_Data.csv')
crime_2015_data = pd.read_csv('data/Annual_Crime_Dataset_2015.csv')
library_data = pd.read_csv('data/Austin_Public_Library_Locations.csv')
water_consumption_data = pd.read_csv('data/Austin_Water_-_Residential_Water_Consumption.csv')
campaign_finance_data = pd.read_csv('data/Campaign_Finance_Data_-_Report_Detail_Dataset.csv')
park_data = pd.read_csv('data/City_of_Austin_Parks_data.csv')
public_art_data = pd.read_csv('data/City_of_Austin_Public_Art_Collection.csv')
public_venue_data = pd.read_csv('data/Creative_Workspaces__Performance_Venues__Galleries___Museums.csv')
ev_charging_data = pd.read_csv('data/Electric_Vehicle_Charging_Network.csv')
restaurant_inspection_data = pd.read_csv('data/Restaurant_Inspection_Scores.csv')
traffic_camera_data = pd.read_csv('data/Traffic_Cameras.csv')

### Clean Restaurant Inspections (Multiple Dates per Restaurant)

In [4]:
restaurant_max_inspection = restaurant_inspection_data.groupby('Restaurant Name', as_index = False)['Inspection Date'].agg('max')

restaurant_inspection = pd.merge(left = restaurant_max_inspection, right = restaurant_inspection_data
                      , how = 'inner'
                      , left_on = ['Restaurant Name', 'Inspection Date']
                      , right_on = ['Restaurant Name', 'Inspection Date'])

restaurant_inspection = restaurant_inspection.drop_duplicates()

### Aggregate Files to Zip Code level

In [11]:
crime_2015 = crime_2015_data.groupby(['GO Location Zip', 'Highest NIBRS/UCR Offense Description']
                        , as_index = False).size()

crime_2016 = crime_2016_data.groupby(['GO Location Zip', 'Highest NIBRS/UCR Offense Description']
                        , as_index = False).size()

ev_charging = pd.DataFrame(ev_charging_data.groupby('Postal Code', as_index = False).size())
restaurant_inspection = restaurant_inspection_data.groupby('Zip Code', as_index = False).agg(['median','size'])
public_art = public_art_data.groupby('Location Zip Code', as_index = False).size()
public_venue = public_venue_data.groupby('ZIP', as_index = False).size()
park = park_data.groupby('ZIP_CODE', as_index = False).size()
water_consumption = water_consumption_data.groupby('Postal Code', as_index = False).size()

### Create List of Austin Zip Codes (to use as base df)

In [6]:
# I'm not sure if this is the right list
austin_zip_codes = {'zip_codes': [78610, 78613, 78617, 78641, 78652, 78653, 78660, 78664, 78681, 78701, 78702, 78703, 78704, 78705
                    , 78712, 78717, 78719, 78721, 78722, 78723, 78724, 78725, 78726, 78727, 78728, 78729, 78730, 78731
                    , 78732, 78733, 78734, 78735, 78736, 78737, 78738, 78739, 78741, 78742, 78744, 78745, 78746, 78747
                    , 78748, 78749, 78750, 78751, 78752, 78753, 78754, 78756, 78757, 78758, 78759]}

austin_zip_codes = pd.DataFrame(austin_zip_codes)

### Combine Files

In [12]:
combined_data = austin_zip_codes.copy()
print(combined_data.shape)

# combined_data = pd.merge(left = combined_data, right = crime_2015_data
#                       , how = 'inner'
#                       , left_on = 'zip_codes'
#                       , right_on = 'GO Location Zip')
# print(combined_data.shape)

# combined_data = pd.merge(left = combined_data, right = crime_2016_data
#                       , how = 'inner'
#                       , left_on = 'zip_codes'
#                       , right_on = 'GO Location Zip')
# print(combined_data.shape)

combined_data = pd.merge(left = combined_data, right = ev_charging
                      , how = 'inner'
                      , left_on = 'zip_codes'
                      , right_on = 'Postal Code')
print(combined_data.shape)

(53, 1)


KeyError: 'Postal Code'